# Tool Calling

In [ ]:
import dotenv
from agents import Agent, ModelSettings, Runner, function_tool, trace

dotenv.load_dotenv()

Create a static calorie table that we can use as a tool:

In [5]:
@function_tool
def get_food_calories(food_item: str) -> str:
    """
    Fetches calorie information for a given food item from a predefined simple calorie database.

    This function accepts the name of a food item and retrieves calorie data if available.
    If the food item is not found in the database, an appropriate message indicating the lack
    of data is returned.

    Parameters:
        food_item: str
            The name of the food item to look up in the calorie database.

    Returns:
        str
            A string containing either the calorie information for the given food item or a message
            indicating that calorie data for the food item is not available.

    Raises:
        None
    """
    # Simple calorie database - in real world, you'd use USDA API
    calorie_data = {
        "apple": "80 calories per medium apple (182g)",
        "banana": "105 calories per medium banana (118g)",
        "broccoli": "25 calories per 1 cup chopped (91g)",
        "almonds": "164 calories per 1oz (28g) or about 23 nuts",
    }

    food_key = food_item.lower()
    if food_key in calorie_data:
        return f"{food_item.title()}: {calorie_data[food_key]}"
    else:
        return f"I don't have calorie data for {food_item} in my database. Try common foods like apple, chicken breast, or rice."

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `get_food_calories` function_

In [8]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    """,
    tools=[get_food_calories],)

In [9]:
with trace("Nutrition Assistant with tools"):
    result = await Runner.run(
        calorie_agent, "How many calories are in total in a banana and an apple?"
    )
    print(result.final_output)

Approximately 185 calories (banana ~105 + apple ~80).


Enforce tools use:

In [ ]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    """,
    tools=[get_food_calories],
    model_settings=ModelSettings(tool_choice="get_food_calories"),
)

with trace("Nutrition Assistant with tools enforced"):
    result = await Runner.run(
        calorie_agent, "How many calories are in total in a banana and an apple?"
    )
    print(result.final_output)